In [ ]:
from ecc import G, S256Point
from helper import encode_base58, hash160, double_sha256

## Define Your Secret (Private Key)

In [ ]:
secret = INSERT_SECRET_HERE

## Create Your Lockbox 


In [ ]:
point = secret * G
if point.y.num % 2 == 1:
    compressed = b'\x03' + point.x.num.to_bytes(32, 'big')
else:
    compressed = b'\x02' + point.x.num.to_bytes(32, 'big')
print("Your lockbox's serial number is: " + "\n" + "\n" + "\"" + compressed.hex() + "\"" + "\n" + "\n" + "This is also know as your public key.")

## Fingerprint Your Lockbox' Serial Number





In [ ]:
sec = bytes.fromhex(compressed.hex())
h160 = hash160(sec)
raw = b"\x6f" + h160
raw = raw + double_sha256(raw)[:4]
addr = encode_base58(raw)
print("Your testnet address: \"" + addr.decode('ascii') + "\"" + "\n")

raw2 = b"\x00" + h160
raw2 = raw2 + double_sha256(raw2)[:4]
addr2 = encode_base58(raw2)
print("Your mainnet address: \"" + addr2.decode('ascii') + "\"")

## Now Go Get Some Magic Internet Money

#### Click here: http://bitcoinfaucet.uo1.net/ copy paste the address that starts with "m", select the highest possible amount and click "Send testnet bitcoins". 

#### Then, wait for at least 1 confirmation. Your can check the testnet block exploer here: 

#### https://live.blockcypher.com/btc-testnet

#### Congratulations! You are now the proud owner of some testnet coins :)

## Let's Spend Our Newly Received Coins

#### In order to do this we will construct a raw transaction and broadcast to the network. 

#### Assume we want to send 40% of the coins that we just received back to the faucet where we just got them from. The other 60% we want to sent back to our own address.

#### Fill in the following items below:

#### 1) The TxID of the transaction you just received in the field that says YOUR_TXID_HERE_
#### 2) The index of your Tx Output in the field that says YOUR_INDEX_HERE (Hint: The first index is 0)
#### 3) Change the target amount to the amount you want to send (YOUR_TARGET_AMOUNT_HERE)
#### 4) Change the change amount to the amount you want to send back to you. (YOUR_CHANGE_AMOUNT_HERE)

#### Note: Fees are implicitly calculated like so: 

###### fees = target_amount - change_amount

#### So make sure that you leave some fees so that your block gets mined. I recommend leaving 0.01 tBTC.


In [ ]:
from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

prev_tx = bytes.fromhex('4fcf01cc213817e9ea0bfc30eeebc8705cf8a0a4ace4b9a4553883144fa229af')
prev_index = 1
target_address = '2NGZrVvZG92qGYqzTLjCAewvPZ7JE8S8VxE'
target_amount = 0.0004
change_address = 'mpc3dAkMqWJFJgQoa7bGMvUsM9eJcTaedD'
change_amount = 0.0005
priv = PrivateKey(secret=secret)

# initialize inputs
tx_ins = []
# create a new tx input with prev_tx, prev_index, blank script_sig and max sequence
tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=prev_index,
            script_sig=Script([]),
            sequence=0xffffffff,
        ))

# initialize outputs
tx_outs = []
# decode the hash160 from the target address
h160 = decode_base58(target_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
# convert target amount to satoshis (multiply by 100 million)
target_satoshis = int(target_amount*100000000)
# create a new tx output for target with amount and script_pubkey
tx_outs.append(TxOut(
    amount=target_satoshis,
    script_pubkey=script_pubkey,
))
# decode the hash160 from the change address
h160 = decode_base58(change_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
# convert change amount to satoshis (multiply by 100 million)
change_satoshis = int(change_amount*100000000)
# create a new tx output for target with amount and script_pubkey
tx_outs.append(TxOut(
    amount=change_satoshis,
    script_pubkey=script_pubkey,
))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
tx_obj.sign_input(0, priv, SIGHASH_ALL)

# SANITY CHECK: change address corresponds to private key
if priv.point.address(testnet=True) != change_address:
    raise RuntimeError('Private Key does not correspond to Change Address, check priv_key and change_address')

# SANITY CHECK: output's script_pubkey is the same one as your address
if tx_ins[0].script_pubkey(testnet=True).items[2] != decode_base58(change_address):
    raise RuntimeError('Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct')

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

### Now, copy the raw transaction and broadcast it to the network by pasting it into the first window here:

https://live.blockcypher.com/btc-testnet/pushtx/

## Well done! You have just send a raw tBTC transaction.